In [1]:
import os
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

from llama_index.core.agent import ReActAgent


In [17]:
import wikipedia
from pydantic import BaseModel
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from typing import Dict

class WikiSearchResult(BaseModel):
    title: str
    url: str

class WikiArticle(BaseModel):
    title: str
    content: str
    url: str

def wikipedia_similar_articles(query: str) -> list[Dict[str,str]]:
    """Search Wikipedia for articles similar to the given query and return titles and URLs."""
    search_results = wikipedia.search(query, results=5)
    result_list = []
    for result in search_results:
        try:
            page = wikipedia.page(result)
            result_list.append(WikiSearchResult(title=page.title, url=page.url))
        except wikipedia.exceptions.DisambiguationError as e:
            # Handle disambiguation pages by logging or ignoring
            print(f"Disambiguation page: {e.options}")
        except wikipedia.exceptions.PageError:
            print(f"PageError: {result}")
    return result_list



def wikipedia_full_article(query: str) -> Dict[str,str]:
    """Fetch the full Wikipedia article for the given query."""
    try:
        page = wikipedia.page(query)
        return WikiArticle(title=page.title, content=page.content, url=page.url)
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle disambiguation pages
        print(f"Disambiguation page: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"PageError: {query}")
    return None

# Wrap these functions in a tool
similar_articles_tool = FunctionTool.from_defaults(fn=wikipedia_similar_articles)
full_article_tool = FunctionTool.from_defaults(fn=wikipedia_full_article)

In [18]:
# llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm = OpenAI(model="gpt-4o")
agent = ReActAgent.from_tools([similar_articles_tool, full_article_tool], llm=llm, verbose=True)

In [8]:

response = agent.chat("Wassssssup")
print(response)


> Running step 0109e6ee-40ad-4e57-a5c3-8af0c9c8210a. Step input: Wassssssup
Thought: The current language of the user is English. The user has greeted me informally. I can respond without using any tools.
Answer: Hey! Not much, just here to help you out. What's up with you?
Hey! Not much, just here to help you out. What's up with you?


In [19]:
response = agent.chat("Tell me about the Eiffel Tower")

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: wikipedia_full_article
Action Input: {'query': 'Eiffel Tower'}
Observation: title='Eiffel Tower' content='The Eiffel Tower (  EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.\nLocally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centerpiece of the 1889 World\'s Fair, and to crown the centennial anniversary of the French Revolution. Although initially criticised by some of France\'s leading artists and intellectuals for its design, it has since become a global cultural icon of France and one of the most recognisable structures in the world.  The tower received 5,889,000 visitors in 2022. The Eiffel Tower is the most visited monument with an entrance fee in the world: 6.9

In [25]:
# Process the response based on which tool was called
if response.sources:  # Check if the response contains sources
    content = []  # List to store the extracted content
    for source in response.sources:
        # Print the tool name that was used
        print(f"Tool used: {source.tool_name}")

        # Each source should contain the ToolOutput with the actual results
        tool_output = source.raw_output

        # Check which tool was used and process the result accordingly
        if isinstance(tool_output, list) and all(isinstance(item, WikiSearchResult) for item in tool_output):
            # This means the similar_articles_tool was used
            print("Similar Articles:")
            for result in tool_output:
                print(f"Title: {result.title}, URL: {result.url}")
                content.append(result.title)  # Append the title to the content list
        
        elif isinstance(tool_output, WikiArticle):
            # This means the full_article_tool was used
            print("Full Article:")
            print(f"Title: {tool_output.title}")
            print(f"Content: {tool_output.content[:500]}...")  # Print the first 500 characters
            print(f"URL: {tool_output.url}")
            content.append(tool_output.content)  # Append the full content to the content list

    # Now `content` should have the extracted content from all sources
    print("Extracted Content:", content)
else:
    print("No tool output was returned.")

print('content: ', content)


Tool used: wikipedia_full_article
Full Article:
Title: Eiffel Tower
Content: The Eiffel Tower (  EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.
Locally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centerpiece of the 1889 World's Fair, and to crown the centennial anniversary of the French Revolution. Although initially criticised by some of France's leading artists a...
URL: https://en.wikipedia.org/wiki/Eiffel_Tower
Extracted Content: ['The Eiffel Tower (  EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.\nLocally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centerpiece of the 1

In [10]:
response = agent.chat("Give me some articles similar to the Eiffel Tower")


> Running step 5733c0c9-8240-401a-bdf6-12169b416592. Step input: Give me some articles similar to the Eiffel Tower
Thought: The current language of the user is English. I need to use a tool to find articles similar to the Eiffel Tower.
Action: wikipedia_similar_articles
Action Input: {'query': 'Eiffel Tower'}
Disambiguation page: ['Eiffel Tower (Paris, Tennessee)', 'Eiffel Tower (Paris, Texas)', 'Eiffel Tower (Six Flags)', 'at Paris Las Vegas', 'Tour Eiffel Bridge', 'Eiffel Tower (Delaunay series)', 'EiffelTowers Nijmegen', 'Heroes Den Bosch', 'Tour Eiffel (disambiguation)', 'List of Eiffel Tower replicas', 'Eiffel (disambiguation)', 'Tower (disambiguation)', 'All pages with titles beginning with Eiffel Tower', 'All pages with titles containing Eiffel Tower']
Observation: [{'title': 'Eiffel Tower', 'url': 'https://en.wikipedia.org/wiki/Eiffel_Tower'}, {'title': 'Eiffel Tower replicas and derivatives', 'url': 'https://en.wikipedia.org/wiki/Eiffel_Tower_replicas_and_derivatives'}, {'titl

In [11]:
response = agent.chat("the school of athens")


> Running step 4c0b29cc-fc85-4564-b4a7-91adaa1838b6. Step input: the school of athens
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: wikipedia_full_article
Action Input: {'query': 'The School of Athens'}
Observation: {'title': 'The School of Athens', 'content': 'The School of Athens (Italian: Scuola di Atene) is a fresco by the Italian Renaissance artist Raphael. It was painted between 1509 and 1511 as part of a commission by Pope Julius II to decorate the rooms now called the Stanze di Raffaello in the Apostolic Palace in Vatican City. \nThe fresco depicts a congregation of ancient philosophers, mathematicians, and scientists, with Plato and Aristotle featured in the center. The identities of most figures are ambiguous or discernable only through subtle details or allusions; among those commonly identified are Socrates, Pythagoras, Archimedes, Heraclitus, Averroes, and Zarathustra. Additionally, Italian artists Leonar

In [12]:
response = agent.chat("Who scored the most goals in the European Champion's League in 2020?")


> Running step 55e8063b-d36d-4c85-bf2e-00cee0520e31. Step input: Who scored the most goals in the European Champion's League in 2020?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: wikipedia_full_article
Action Input: {'query': '2020 UEFA Champions League top scorers'}
PageError: 2020 UEFA Champions League top scorers
Observation: None
> Running step 80333f5c-aa0f-4d31-a7b6-ec572d871897. Step input: None
Thought: It seems there was no direct article found for the specific query. I will try to find similar articles related to the top scorers in the UEFA Champions League for 2020.
Action: wikipedia_similar_articles
Action Input: {'query': '2020 UEFA Champions League top scorers'}
Observation: [{'title': 'List of UEFA Champions League top scorers', 'url': 'https://en.wikipedia.org/wiki/List_of_UEFA_Champions_League_top_scorers'}, {'title': 'List of UEFA Champions League top scorers', 'url': 'https://en.wikipedia.org/wiki/

In [13]:
print(response)

The top scorer in the UEFA Champions League for the 2019-2020 season was Robert Lewandowski of Bayern Munich, who scored 15 goals.


In [14]:
response = agent.chat("What about in 2020-2021?")


> Running step 016928f6-fb7d-4478-8bd2-fa2fffc25989. Step input: What about in 2020-2021?
Thought: (Implicit) I can answer without any more tools!
Answer: The top scorer in the UEFA Champions League for the 2020-2021 season was Erling Haaland of Borussia Dortmund, who scored 10 goals.


In [15]:
agent.reset()
response = agent.chat("Who's the president?")

> Running step a560bc54-c21d-4729-886f-12a8ec1143a9. Step input: Who's the president?
Thought: The current language of the user is English. I need to use a tool to find out the current president.
Action: wikipedia_full_article
Action Input: {'query': 'President of the United States'}
Observation: {'title': 'President of the United States', 'content': 'The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.\nThe power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansions during the presidencies of F

In [16]:
agent.reset()
response = agent.chat("css flex")

> Running step baa0b490-76aa-445f-8a6f-cdcfccd0bc41. Step input: css flex
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: wikipedia_similar_articles
Action Input: {'query': 'CSS Flex'}
Observation: [{'title': 'CSS Flexible Box Layout', 'url': 'https://en.wikipedia.org/wiki/CSS_Flexible_Box_Layout'}, {'title': 'Light-emitting diode', 'url': 'https://en.wikipedia.org/wiki/Light-emitting_diode'}, {'title': 'CSS hack', 'url': 'https://en.wikipedia.org/wiki/CSS_hack'}, {'title': 'NetSurf', 'url': 'https://en.wikipedia.org/wiki/NetSurf'}, {'title': 'Holy grail (web design)', 'url': 'https://en.wikipedia.org/wiki/Holy_grail_(web_design)'}]
> Running step 986b58e6-382b-4e21-aba0-68784d79696d. Step input: None
Thought: The most relevant article to the user's query is "CSS Flexible Box Layout". I will fetch the full article for more detailed information.
Action: wikipedia_full_article
Action Input: {'query': 'CSS Flexible Box Lay

In [17]:
agent.reset()
response = agent.chat("articles similar to the article on 'Philosophy'?")

> Running step 439ace7d-d61d-4c6a-ac88-c171cdac789a. Step input: articles similar to the article on 'Philosophy'?
Thought: The current language of the user is English. I need to use a tool to find articles similar to the article on 'Philosophy'.
Action: wikipedia_similar_articles
Action Input: {'query': 'Philosophy'}
Observation: [{'title': 'Philosophy', 'url': 'https://en.wikipedia.org/wiki/Philosophy'}, {'title': 'Western philosophy', 'url': 'https://en.wikipedia.org/wiki/Western_philosophy'}, {'title': 'Contemporary philosophy', 'url': 'https://en.wikipedia.org/wiki/Contemporary_philosophy'}, {'title': 'Will (philosophy)', 'url': 'https://en.wikipedia.org/wiki/Will_(philosophy)'}, {'title': 'Hindu philosophy', 'url': 'https://en.wikipedia.org/wiki/Hindu_philosophy'}]
> Running step 66a17afb-9201-4070-bff1-b70e53084ec8. Step input: None
Thought: I have found several articles similar to the article on 'Philosophy'. I will list them for the user.
Answer: Here are some articles similar 

In [18]:
agent.reset()
response = agent.chat("tell me a joke")

> Running step 37302942-e34a-46a7-9057-01d3c663de60. Step input: tell me a joke
Thought: The current language of the user is English. I can answer without using any tools.
Answer: Why don't scientists trust atoms?

Because they make up everything!


In [19]:

prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may

In [20]:
from llama_index.core.workflow import draw_all_possible_flows

draw_all_possible_flows(ReActAgent, filename="react_agent_workflow_2.html")

/tmp/ipykernel_18247/3755135375.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) draw_all_possible_flows. (Install `llama-index-utils-workflow` and use the import `from llama_index.utils.workflow` instead.)
  draw_all_possible_flows(ReActAgent, filename="react_agent_workflow_2.html")


react_agent_workflow_2.html
